In [ ]:
import pandas as pd
import numpy as np
import string
import csv
import urllib

In [ ]:
import os

# modify the line below and add the path to your google credentials json file
# further information on how to generate such a json file: https://cloud.google.com/docs/authentication/getting-started#command-line
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = PLACEHOLDER_APPLICATION_CREDENTIALS

In [ ]:
from google.cloud import storage

# first, create a new bucket on Google Cloud Storage: https://cloud.google.com/storage/docs/creating-buckets
# upload the videos in this bucket
# to list all the videos in the bucket, use the function below

def list_blobs_with_prefix(bucket_name, prefix, delimiter):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix=prefix, delimiter=delimiter)

    videos = []
    for blob in blobs:
        if not blob.name.endswith("/"):
            videos.append("gs://" + bucket_name + "/" + blob.name)
            
    return videos
            
videos = list_blobs_with_prefix(PLACEHOLDER_BUCKET_NAME, PLACEHOLDER_PREFIX, PLACEHOLDER_DELIMITER)

In [ ]:
"""Transcribe speech from a video stored on GCS."""
from google.cloud import videointelligence
import os

from google.cloud import storage
import csv

# Replace all PLACEHOLDER_ variables with the variables you have created.
gcloud_bucket_name = PLACEHOLDER_BUCKET_NAME_FOR_SUBTITLES
storage_client = storage.Client()
bucket = storage_client.get_bucket(gcloud_bucket_name)

names = ["storageLink","alternative_no","transcript","confidence","start_time","end_time","text"]

for video in videos:

    videoTitle = video.split("/")[-1]
    
    with open("../data/video_subtitles/" + videoTitle + ".csv", "w" ) as outSentences:
        writer = csv.writer( outSentences )
        writer.writerow(names)


    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.enums.Feature.SPEECH_TRANSCRIPTION]

    config = videointelligence.types.SpeechTranscriptionConfig(
            language_code='en-US',
            enable_automatic_punctuation=True)
    video_context = videointelligence.types.VideoContext(
            speech_transcription_config=config)

    operation = video_client.annotate_video(
            video, features=features,
            video_context=video_context)

    print('\nProcessing video for speech transcription.')

    result = operation.result(timeout=3600)

    # There is only one annotation_result since only
    # one video is processed.
    annotation_results = result.annotation_results[0]
    #print(annotation_results)
    for speech_transcription in annotation_results.speech_transcriptions:
        #print(speech_transcription)
        # The number of alternatives for each transcription is limited by
        # SpeechTranscriptionConfig.max_alternatives.
        # Each alternative is a different possible transcription
        # and has its own confidence score.
        #print(speech_transcription.alternatives)
        #print(len(speech_transcription.alternatives))
        index = 0
        for alternative in speech_transcription.alternatives:

            print('Alternative level information:')

            #print('Transcript: {}'.format(alternative.transcript))
            #print('Confidence: {}\n'.format(alternative.confidence))

            print('Word level information:')
            for word_info in alternative.words:
                word = word_info.word
                start_time = word_info.start_time
                end_time = word_info.end_time
                print('\t{}s - {}s: {}'.format(
                    start_time.seconds + start_time.nanos * 1e-9,
                    end_time.seconds + end_time.nanos * 1e-9,
                    word))
                row = []
                row.append(videoTitle)
                row.append(index)
                row.append(alternative.transcript)
                row.append(alternative.confidence)
                row.append(start_time.seconds + start_time.nanos * 1e-9)
                row.append(end_time.seconds + end_time.nanos * 1e-9)
                row.append(word)

                with open("../data/video_subtitles/" + videoTitle + ".csv", "a" ) as outSentences:
                    writer = csv.writer( outSentences )
                    writer.writerow(row)
            index = index + 1

    blob = bucket.blob("" + videoTitle + ".csv")
    blob.upload_from_filename("../data/video_subtitles/" + videoTitle + ".csv")